In [2]:
import pandas as pd
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [4]:
# reader = PdfReader("sample.pdf")
# number_of_pages = len(reader.pages)
# page = reader.pages[0]
# text = page.extract_text()
# print(text)

fileName = "sample.pdf"

def readPDF(name):

    loader = PyPDFLoader(name)
    pages = loader.load_and_split()
    return pages

doc = readPDF(fileName)
print(doc)

[Document(metadata={'source': 'sample.pdf', 'page': 0}, page_content='C H A R L E S  M C T U R L A N D\nS O F T W A R E  E N G I N E E R\nC O N T A C T\ncmcturland@email.com\n(123) 456-7890\nNew York, NY\nLinkedIn\nE D U C A T I O N\nB.S.\nComputer Science\nUniversity of Pittsburgh\nSeptember 2008 - April 2012\nPittsburgh, PA\nS K I L L S\nPython (Django)\nJavascript (NodeJS ReactJS,\njQuery)\nSQL (MySQL, PostgreSQL,\nNoSQL)\nHTML5/CSS\nAWS\nUnix, GitW O R K  E X P E R I E N C E\nSoftware Engineer\nEmbark\nJanuary 2015 - current/New York, NY\nWorked with product managers to re-architect a multi-page web\napp into a single page web-app, boosting yearly revenue by $1.4M\nConstructed the logic for a streamlined ad-serving platform that\nscaled to our 35M users, which improved the page speed by 15%\nafter implementation\nTested software for bugs and operating speed, ﬁxing bugs and\ndocumenting processes to increase efﬁciency by 18%\nIterated platform for college admissions, collaborating w

In [ ]:
def chunk_data(doc,chunk_size=1000,chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size,chunk_overlap)
    chunks = text_splitter.split_documents(doc)
    return chunks

chunk_data()

In [2]:
data = pd.read_csv('./Resume.csv')
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
